#### Домашнее задание.
## Урок 8 - Функционал Time Series
#### Выполнил работу: Данил Кузнецов
Посчитайте Retention по когортам даты установки.
Размер когорты — неделя.
Вам будет дано 2 файла:
© geekbrains.ru 7
1. С событиями установок приложения пользователями (install).
2. С событиями входа в игру (session_start).

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
installs = pd.read_csv('les8_installs.csv',index_col = 0)

### Переведу формат

In [3]:
installs['install_date'] = pd.to_datetime(installs['install_date'], format='%Y-%m-%d')

### Вычисляю начало недели когорты

In [4]:
installs['week_start'] = installs['install_date'].dt.to_period('W').dt.start_time

### Читаю второй файл

In [5]:
logins = pd.read_csv('les8_logins.csv',index_col = 0)
logins.reset_index(drop = True)

,client_id,event_timestamp,is_tester,is_youtuber
0,868610407601689670,2021-02-16 07:53:04,0,0
1,2434015632087848193,2021-02-16 07:51:23,0,0
2,2531632462363212905,2021-02-16 07:52:12,0,0
3,3205304356447992721,2021-02-16 07:54:17,0,0
4,7482763148151863115,2021-02-16 07:54:36,0,0
...,...,...,...,...
286173,374127069099732578,2021-02-15 12:56:25,0,0
286174,4905473983468376225,2021-02-15 12:57:54,0,0
286175,110174818069209226,2021-02-15 12:58:05,0,0
286176,590901015821838791,2021-02-15 12:58:33,0,0


### Перевожу формат

In [6]:
logins['event_timestamp'] = pd.to_datetime(logins['event_timestamp'], format='%Y-%m-%d')

### Склеиваю таблицы

In [7]:
df = installs.merge(logins, on='client_id', how='right')

### Вычисляю день жизни

In [8]:
df['lifetime'] = df['event_timestamp'] - df['install_date']
df['lifetime'] = df['lifetime'] / np.timedelta64(1, 'D')
df['lifetime'] = df['lifetime'].astype('int')
df

,client_id,install_date,week_start,event_timestamp,is_tester,is_youtuber,lifetime
0,868610407601689670,2021-01-06,2021-01-04,2021-02-16 07:53:04,0,0,41
1,2434015632087848193,2021-01-17,2021-01-11,2021-02-16 07:51:23,0,0,30
2,2531632462363212905,2021-01-07,2021-01-04,2021-02-16 07:52:12,0,0,40
3,3205304356447992721,2021-01-15,2021-01-11,2021-02-16 07:54:17,0,0,32
4,7482763148151863115,2021-01-13,2021-01-11,2021-02-16 07:54:36,0,0,34
...,...,...,...,...,...,...,...
286173,374127069099732578,2021-01-25,2021-01-25,2021-02-15 12:56:25,0,0,21
286174,4905473983468376225,2021-01-07,2021-01-04,2021-02-15 12:57:54,0,0,39
286175,110174818069209226,2021-01-25,2021-01-25,2021-02-15 12:58:05,0,0,21
286176,590901015821838791,2021-01-04,2021-01-04,2021-02-15 12:58:33,0,0,42


### Группирую и агрегирую

In [9]:
final_table = df.groupby(['week_start','lifetime']).agg({'client_id':'nunique'})
final_table.columns = ['clients']

### Добавляю столбец о кол-ве пользователей в когорте

In [10]:
installers = df.groupby(['week_start']).agg({'client_id':'nunique'})
installers.columns =['clients_in_cohort']
installers

,clients_in_cohort
week_start,
2020-12-28,2835
2021-01-04,7382
2021-01-11,7013
2021-01-18,6771
2021-01-25,4423


In [11]:

final_table = final_table.join(installers, on='week_start', how='left')
final_table

clients  clients_in_cohort
week_start lifetime                            
2020-12-28 0            2835               2835
           1             936               2835
           2             592               2835
           3             470               2835
           4             406               2835
...                      ...                ...
2021-01-25 107            27               4423
           108            18               4423
           109            21               4423
           110            13               4423
           111             7               4423

[627 rows x 2 columns]

### Добавляю столбец о Retention

In [14]:
final_table['Retention'] = final_table['clients'] / final_table['clients_in_cohort']

In [15]:
final_table

clients  clients_in_cohort  Retention
week_start lifetime                                       
2020-12-28 0            2835               2835   1.000000
           1             936               2835   0.330159
           2             592               2835   0.208818
           3             470               2835   0.165785
           4             406               2835   0.143210
...                      ...                ...        ...
2021-01-25 107            27               4423   0.006104
           108            18               4423   0.004070
           109            21               4423   0.004748
           110            13               4423   0.002939
           111             7               4423   0.001583

[627 rows x 3 columns]